In [5]:
import numpy as np
import PIL
from PIL import Image
import cv2


In [ ]:
# https://bottosson.github.io/posts/oklab/

M1 = np.array([
    [0.4122214708, 0.5363325363, 0.0514459929],
    [0.2119034982, 0.6806995451, 0.1073969566],
    [0.0883024619, 0.2817188376, 0.6299787005]
])

M2 = np.array([
    [0.2104542553, 0.7936177850, -0.0040720468],
    [1.9779984951, -2.4285922050, 0.4505937099],
    [0.0259040371, 0.7827717662, -0.8086757660]
])



# M1_inv = np.linalg.inv(M1)
# M2_inv = np.linalg.inv(M2)


def linear_to_oklab(rgb: np.array):
    lms = np.einsum('ijk,lk', rgb, M1)
    lms_ = np.cbrt(lms)
    return np.einsum('ijk,lk', lms_, M2)

def oklab_to_linear(oklab: np.array):
    lms = np.einsum('ijk,lk', oklab, M2_inv)
    lms_ = np.power(lms, 3)
    linear = np.einsum('ijk,lk', lms_, M1_inv)
    # Clamp negative elements
    return np.max(linear, 0.0)

def oklab_to_oklch(oklab: np.array):
    L, a, b = oklab[..., 0], oklab[..., 1], oklab[..., 2]
    C = np.sqrt(np.power(a, 2) + np.power(b, 2))
    H = np.arctan2(b, a)
    return np.stack([L, C, H], axis=-1)

def oklch_to_oklab(oklch: np.array):
    L, C, H = oklch[..., 0], oklch[..., 1], oklch[..., 2]
    a = C * np.cos(H)
    b = C * np.sin(H)
    return np.stack([L, a, b], axis=-1)

def srgb_to_linear(srgb: np.array) -> np.array:
    gamma = ((srgb + 0.055) / 1.055)**2.4
    scale = srgb / 12.92
    return np.where (srgb > 0.04045, gamma, scale)

def linear_to_srgb(linear: np.array) -> np.array:
    scale = 12.92 * linear
    gamma = 1.055 * np.power(linear, 1/2.4) - 0.055
    return np.where(linear > 0.0031308, gamma, scale)

def srgb_to_oklch(srgb: np.array):
    return oklab_to_oklch(linear_to_oklab(srgb_to_linear(srgb)))

def oklch_to_srgb(lch: np.array):
    return linear_to_srgb(oklab_to_linear(oklch_to_oklab(lch)))




pil_image = Image.open('../original.png')
image = np.array(pil_image, np.float32) / 255.0

image_xyz = cv2.cvtColor(image, cv2.COLOR_RGB2XYZ)
print(image_xyz[400, 400, :])
# oklch_image = srgb_to_oklch(image)
# hue_shift = np.random.uniform(-20 * np.pi / 180.0, 20 * np.pi / 180.0)
# oklch_image[..., 2] += hue_shift
# image = oklch_to_srgb(oklch_image)

pil_image: PIL.Image = Image.open('../original.png')


In [ ]:

image = np.array(pil_image, np.uint8).astype(np.float32) / 255.0
linear_image = srgb_to_linear(image)

oklab_image = linear_to_oklab(linear_image)
oklch_image = oklab_to_oklch(oklab_image)




hue_shift = -40.0 * np.pi / 180.0
oklch_image[..., 2] += hue_shift
oklch_image[..., 2] = np.mod(oklch_image[..., 2], 2.0 * np.pi)


oklch_out = oklch_to_oklab(oklch_image)
linear_out = oklab_to_linear(oklch_out)


srgb_out = linear_to_srgb(linear_out)


print(np.where(linear_out < 0))
print(linear_out[750, 609, :])
print(srgb_out[750, 609, :])


image = np.clip(srgb_out * 255, 0, 255).astype(np.uint8)
Image.fromarray(image)



In [ ]:

image = np.array(pil_image, np.uint8).astype(float) / 255.0
oklab_image = linear_to_oklab(linear_image)
oklch_image = oklab_to_oklch(oklab_image)

# hue_shift = np.random.uniform(-40 * np.pi / 180.0, 20 * np.pi / 180.0)
hue_shift = -40 * np.pi / 180.0
oklch_image[..., 2] += hue_shift

oklab_image = oklch_to_oklab(oklch_image)
if np.any(oklab_image < 0):
    print("Negative values in oklab")
linear_image = oklab_to_linear(oklab_image)
if np.any(linear_image < 0):
    print("Negative values in linear")
srgb_image = linear_to_srgb(linear_image)


image = oklch_to_srgb(oklch_image)
print(hue_shift * 180.0 / np.pi)



image = np.clip(image * 255, 0, 255).astype(np.uint8)
Image.fromarray(image)